# Almacenamiento de Agua

Importar librerías a usar.

- Numpy
- ee: API de Google Earth Engine
- folium: visualización de mapas sobre Leaflet

In [105]:
import numpy as np
import ee
import folium
import pandas as pd

Inicializar sesión de Google Earth Engine

In [106]:
ee.Initialize()

Parámetros de visualización (composición de bandas) para Sentinel-2

In [107]:
visRGB = {"bands": ["B4","B3","B2"],"min":0,"max":2000}
visRGB_water = {"bands": ["B4","B3","B2"],"min":0,"max":800}
visIndex = {"min":0,"max":1}

## Datos iniciales

### Fecha

- interestDate: Fecha deseada para la cual se desea calcular el volumen de agua almacenada.
- deltaDays: Rango de fechas para buscar imágenes adicionales +/- la fecha de interés.

In [108]:
interestDate = "2019-10-22"
deltaDays = 4

### Región

- xmin: longitud mínima.
- xmax: longitud máxima.
- ymin: latitud mínima.
- ymax: latitud máxima.

In [109]:
xmin = -8.22603391725042
ymin = 41.85962828770244
xmax = -8.063298931898858
ymax = 41.93092895284894

In [110]:
centerx = np.array([xmin,xmax]).mean()
centery = np.array([ymin,ymax]).mean()

In [111]:
ROI = ee.Geometry.Rectangle([xmin,ymin,xmax,ymax])

## Funciones

### 1. Visualización de una imagen en Folium

In [112]:
def foliumLayer(image,parameters = visRGB,layer_name = "layer"):
    
    folium_map = folium.Map(location = [centery,centerx],zoom_start = 13)
    
    mapIdDict = image.getMapId(parameters) # convertir imagen a id de visualizacion
    
    tile = folium.TileLayer(tiles = mapIdDict['tile_fetcher'].url_format,
                            attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
                            overlay = True,
                            name = layer_name)
    
    tile.add_to(folium_map)
    
    folium_map.add_child(folium.LayerControl())
    
    return folium_map

### 2. Cortar imágenes por ROI

In [113]:
def clip_images(image):
        
    return image.clip(ROI).copyProperties(image,["system:time_start"]) # retornar imagenes recortadas

### 3. Enmascarar nubes y sombras de una imagen

In [114]:
def clouds_shadows_mask(image):
    
    shadows_mask = image.select('SCL').eq(3).Not() # pixeles que no son sombra
    clouds_mask = image.select('SCL').lt(7).Or(image.select('SCL').gt(9)) # pixeles que no son nubes
    mask = shadows_mask.And(clouds_mask) # pixeles que no son ni sombra ni nubes
    
    return image.updateMask(mask).copyProperties(image,["system:time_start"]) # retornar imagenes enmascaradas

### 4. Seleccionar imágenes

In [115]:
def collectS2Images(interestDate,deltaDays,xmin,ymin,xmax,ymax,clipImages = True):
    
    ROI = ee.Geometry.Rectangle([xmin,ymin,xmax,ymax])
    
    interestDate = np.datetime64(interestDate)
    initialDate = np.datetime_as_string(interestDate - np.timedelta64(deltaDays,'D'))
    finalDate = np.datetime_as_string(interestDate + np.timedelta64(deltaDays,'D'))
    
    IC = ee.ImageCollection("COPERNICUS/S2_SR").filterDate(initialDate,finalDate).filterBounds(ROI)
    
    if clipImages:
        
        def clip_images(image):        
            return image.clip(ROI).copyProperties(image,["system:time_start"]) # retornar imagenes recortadas
        
        IC = IC.map(clip_images)
    
    return IC

## Paso 1. Seleccionar imágenes

In [116]:
S2 = collectS2Images(interestDate,deltaDays,xmin,ymin,xmax,ymax)

In [117]:
foliumLayer(S2.first(),layer_name = "Imagen sin mascara")

## Paso 2. Enmascarar nubes y sombras

In [118]:
S2_masked = S2.map(clouds_shadows_mask)

In [119]:
foliumLayer(S2_masked.first(),layer_name = "Imagen con mascara")

## Paso 3. Reducir imagen

In [120]:
S2_reduced = S2_masked.median()

In [121]:
foliumLayer(S2_reduced,layer_name = "Imagen reducida")

## Paso 4. Calcular Índice de Vegetación

In [122]:
VI = S2_reduced.normalizedDifference(['B8','B3'])

In [123]:
foliumLayer(VI,visIndex,"Index")

## Paso 5. Segmentación

In [124]:
seeds = ee.Algorithms.Image.Segmentation.seedGrid(20)

SNIC = ee.Algorithms.Image.Segmentation.SNIC(image = ee.Image.cat([S2_reduced.select(['B2','B3','B4','B8']),VI]),
                                             size = 32,
                                             compactness = 1,
                                             connectivity = 8,
                                             neighborhoodSize = 256,
                                             seeds = seeds)

SNIC = SNIC.select(['B2_mean','B3_mean','B4_mean','B8_mean','nd_mean','clusters'], ['B2','B3','B4','B8','VI','clusters'])

In [125]:
foliumLayer(SNIC.select("clusters").randomVisualizer(),{},"Clusters")

## Paso 6. Clusterizar objetos por K-means

In [126]:
objectPropertiesImage = SNIC.select(['B2','B3','B4','B8','VI'])

X_train = objectPropertiesImage.sample(scale = 10,numPixels = 5000,region = ROI)

k = 3

kmeans = ee.Clusterer.wekaKMeans(k)
kmeans = kmeans.train(X_train)

clusterImage = objectPropertiesImage.cluster(kmeans)

In [127]:
foliumLayer(clusterImage.randomVisualizer(),{},"Kmeans")

## Paso 7. Seleccionar cluster de agua

In [128]:
kmeans.schema().getInfo()

['B2', 'B3', 'B4', 'B8', 'VI']

In [129]:
values = []

for i in range(k):
    cluster_mask = clusterImage.eq(i)
    VI_clusterMasked = VI.updateMask(cluster_mask)
    mean_value = VI_clusterMasked.reduceRegion(reducer = ee.Reducer.mean(),geometry = ROI,scale = 50)
    values.append(mean_value.getInfo()['nd'])  

In [130]:
cluster_water = np.array(values).argmin().item()

In [131]:
water_mask = clusterImage.eq(cluster_water)
water = S2_reduced.updateMask(water_mask)

In [132]:
foliumLayer(water,visRGB_water,"Agua clasificada")

### Estimación de Batimetría

In [133]:
depth = water.select("B2").log().divide(water.select("B3").log())

In [134]:
foliumLayer(depth,{},"Depth")

In [135]:
kernel = ee.Kernel.square(radius = 2,normalize = False)

depth_median = depth.focal_mean(kernel = kernel,iterations = 1).updateMask(water_mask)

In [136]:
foliumLayer(depth_median,{},"Depth Median")

In [137]:
bathyPath = "C:/Users/Dave Mont/Desktop/Master_of_DataScience/TFM/bathymetric_data"
bathyPathtxt = "C:/Users/Dave Mont/Desktop/Master_of_DataScience/TFM/bathymetric_data_wo_header.txt"

f = open(bathyPath)
textList = f.readlines()[7:]

outF = open(bathyPathtxt, "w")
for line in textList:
    line = line.replace(",",".")
    outF.write(line)    
outF.close()

In [138]:
bathy = np.loadtxt(bathyPathtxt,delimiter = "\t",usecols = (2,3,5))

In [166]:
water = water.addBands(water.pixelLonLat())

In [202]:
imageToReduce = water.select(['B2','B3','B4','B8','longitude','latitude'])

In [206]:
extractedData = []

k = 0

print("Comenzando la extracción de datos...")
while k <= bathy.shape[0]:
    
    print("******************************")
    print("Creando nuevo batch...")
    pointFeatures = []
        
    initial = k
    print("Inicia en",initial)
    
    if k + 5000 > bathy.shape[0]:
        final = bathy.shape[0]
    else:
        final = k + 5000
    print("Finaliza en",final)
        
    print("Realizando extracción...")
    for i in range(initial,final):
        pointFeatures.append(ee.Geometry.Point([bathy[i,0],bathy[i,1]]))
        
    fromList = ee.FeatureCollection(pointFeatures)
    
    imageDictionary = imageToReduce.reduceRegions(collection = fromList,reducer = ee.Reducer.first(),scale = 10)
    
    features = imageDictionary.getInfo()['features']
    
    for i in range(len(features)):
        extractedData.append(list(features[i]['properties'].values()))
    
    print("Extracción finalizada")
    print("Avance:",final*100/bathy.shape[0],"%")
    
    k = k + 5000

Comenzando la extracción de datos...
******************************
Creando nuevo batch...
Inicia en 0
Finaliza en 5000
Realizando extracción...
Extracción finalizada
Avance: 29.684160531940158 %
******************************
Creando nuevo batch...
Inicia en 5000
Finaliza en 10000
Realizando extracción...
Extracción finalizada
Avance: 59.368321063880316 %
******************************
Creando nuevo batch...
Inicia en 10000
Finaliza en 15000
Realizando extracción...
Extracción finalizada
Avance: 89.05248159582047 %
******************************
Creando nuevo batch...
Inicia en 15000
Finaliza en 16844
Realizando extracción...
Extracción finalizada
Avance: 100.0 %


In [205]:
extractedData

[[104, 225, 164, 205, 41.920646301428974, -8.084253652141015],
 [104, 225, 164, 205, 41.920646301428974, -8.084253652141015],
 [104, 225, 164, 205, 41.920646301428974, -8.084253652141015],
 [104, 225, 164, 205, 41.920646301428974, -8.084253652141015],
 [104, 225, 164, 205, 41.920646301428974, -8.084253652141015],
 [104, 225, 164, 205, 41.920646301428974, -8.084253652141015],
 [104, 225, 164, 205, 41.920646301428974, -8.084253652141015],
 [104, 225, 164, 205, 41.920646301428974, -8.084253652141015],
 [104, 225, 164, 205, 41.920646301428974, -8.084253652141015],
 [104, 225, 164, 205, 41.920646301428974, -8.084253652141015],
 [104, 225, 164, 205, 41.920646301428974, -8.084253652141015],
 [104, 225, 164, 205, 41.920646301428974, -8.084253652141015],
 [104, 225, 164, 205, 41.920646301428974, -8.084253652141015],
 [104, 225, 164, 205, 41.920646301428974, -8.084253652141015],
 [104, 225, 164, 205, 41.920646301428974, -8.084253652141015],
 [104, 225, 164, 205, 41.920646301428974, -8.0842536521

In [189]:
meanDictionary = imageToReduce.reduceRegions(collection = fromList,reducer = ee.Reducer.first(),scale = 10)

In [190]:
features = meanDictionary.getInfo()['features']

In [191]:
list(features[i]['properties'].keys())

['B2', 'B3', 'B4', 'B8', 'latitude', 'longitude']

In [192]:
extractedData = []
for i in range(len(features)):
    extractedData.append(list(features[i]['properties'].values()))

In [193]:
extractedData

[[104, 225, 164, 205, 41.920646301428974, -8.084253652141015],
 [104, 225, 164, 205, 41.920646301428974, -8.084253652141015],
 [104, 225, 164, 205, 41.920646301428974, -8.084253652141015],
 [104, 225, 164, 205, 41.920646301428974, -8.084253652141015],
 [104, 225, 164, 205, 41.920646301428974, -8.084253652141015],
 [104, 225, 164, 205, 41.920646301428974, -8.084253652141015],
 [104, 225, 164, 205, 41.920646301428974, -8.084253652141015],
 [104, 225, 164, 205, 41.920646301428974, -8.084253652141015],
 [104, 225, 164, 205, 41.920646301428974, -8.084253652141015],
 [104, 225, 164, 205, 41.920646301428974, -8.084253652141015],
 [104, 225, 164, 205, 41.920646301428974, -8.084253652141015],
 [104, 225, 164, 205, 41.920646301428974, -8.084253652141015],
 [104, 225, 164, 205, 41.920646301428974, -8.084253652141015],
 [104, 225, 164, 205, 41.920646301428974, -8.084253652141015],
 [104, 225, 164, 205, 41.920646301428974, -8.084253652141015],
 [104, 225, 164, 205, 41.920646301428974, -8.0842536521

In [160]:
list(features[0]['properties'].keys())

['B2', 'B3', 'B4', 'B8']

In [74]:
meanDictionary = water.reduceRegion(reducer = ee.Reducer.first(),geometry = multiPoint,scale = 10)